In [6]:
import numpy as np
import requests
import pandas as pd
from IPython.display import display, HTML

In [7]:
import sys
import os
import time
import re
import json
from urllib.parse import quote as urlencode
from urllib.request import urlretrieve
import http.client as httplib 
from astropy.table import Table

In [8]:
tce_csv = "./data/tce_stats/tess2018206190142-s0001-s0009_dvr-tcestats.csv"

In [9]:
tce_data = pd.read_csv(tce_csv, skiprows=6)

In [42]:
tce_data.head(50)

,tceid,ticid,planetNumber,planetCandidateCount,starTeffKelvin,starLoggCgs,orbitalPeriodDays,orbitalPeriodDays_err,planetRadiusEarthRadii,planetRadiusEarthRadii_err,...,transitIngressTimeHours,minImpactParameter,ws_mes,ws_mesphase,expectedtransitcount,chiSquare2,mes,maxses,equilibriumTempKelvin,InsolationFlux
0,00101363142-01,101363142,1,1,3237.0,4.99853,28.576300,0.002830,3.712010,0.924963,...,0.219546,0.029692,6.08604,10.00830,2,3.67798,14.35090,12.95090,214.556,0.500820
1,00101364075-01,101364075,1,1,5758.0,4.40457,24.406100,0.016645,8.375130,6.407170,...,7.839750,0.990105,2.75692,3.12639,2,0.72868,8.25559,6.32646,639.778,39.594700
2,00101365484-01,101365484,1,1,3536.0,4.43800,29.930900,0.004922,1.364970,1.170960,...,0.079310,0.068416,5.20653,-0.64444,2,11.91730,9.87100,7.93837,317.564,2.403520
3,00101395259-01,101395259,1,1,6422.0,4.20608,7.574500,0.000261,13.542500,3.806770,...,0.905589,0.878173,3.17116,3.72500,6,1.63997,70.55390,34.21930,1204.170,496.905000
4,00101395750-01,101395750,1,3,8596.0,3.66834,0.247308,0.000015,3.734160,0.875448,...,0.009423,0.699687,10.57780,0.14896,207,96.69240,9.21249,2.80119,0.000,0.000000
5,00101395750-02,101395750,2,3,8596.0,3.66834,0.247269,0.000010,4.344770,1.331550,...,0.005610,0.302115,9.76332,0.14832,207,91.55760,9.24523,2.91435,0.000,0.000000
6,00101395750-03,101395750,3,3,8596.0,3.66834,0.247232,0.000011,3.920140,1.808480,...,0.005118,0.300000,8.05357,0.14792,207,157.37200,10.05510,4.06208,0.000,0.000000
7,00101417386-01,101417386,1,3,8439.0,4.13249,15.504900,0.002712,4.799780,0.992296,...,0.108761,0.435707,5.15105,2.23611,3,0.01832,7.73992,5.49239,1353.310,792.713000
8,00101417386-02,101417386,2,3,8439.0,4.13249,17.717700,0.002787,4.879090,0.885955,...,0.130197,0.503774,6.14138,-2.17917,3,0.01099,7.64800,5.44563,1294.450,663.535000
9,00101417386-03,101417386,3,3,8439.0,4.13249,30.155200,0.005019,4.776290,0.771390,...,0.337603,0.858589,5.39371,19.08750,2,0.00003,7.10557,5.03160,1084.180,326.530000


In [10]:
len(tce_data)

3889

In [11]:
tessids = []
for tessid in tce_data['ticid']:
    if tessid not in tessids:
        tessids.append(tessid)

In [12]:
tceids = []
for tce in tce_data['tceid']:
    if tce not in tceids:
        tceids.append(tce)

In [37]:
# I think the missing columns can ge gotten from astroquery relatively easily
tce_data.columns

Index(['tceid', 'ticid', 'planetNumber', 'planetCandidateCount',
       'starTeffKelvin', 'starLoggCgs', 'orbitalPeriodDays',
       'orbitalPeriodDays_err', 'planetRadiusEarthRadii',
       'planetRadiusEarthRadii_err', 'ratioPlanetRadiusToStarRadius',
       'ratioSemiMajorAxisToStarRadius', 'semiMajorAxisAu',
       'starRadiusSolarRadii', 'transitDepthPpm', 'transitDepthPpm_err',
       'transitDurationHours', 'transitEpochBtjd', 'transitEpochBtjd_err',
       'transitIngressTimeHours', 'minImpactParameter', 'ws_mes',
       'ws_mesphase', 'expectedtransitcount', 'chiSquare2', 'mes', 'maxses',
       'equilibriumTempKelvin', 'InsolationFlux'],
      dtype='object')

In [54]:
tce_data.loc[tce_data['ticid'] == 101395750]['starRadiusSolarRadii'].values[0]

3.5552900000000003

In [45]:
tessids[10]

118316147

In [13]:
def mastQuery(request):
    """Perform a MAST query.
    
        Parameters
        ----------
        request (dictionary): The MAST request json object
        
        Returns head,content where head is the response HTTP headers, and content is the returned data"""
    
    server='mast.stsci.edu'

    # Grab Python Version 
    version = ".".join(map(str, sys.version_info[:3]))

    # Create Http Header Variables
    headers = {"Content-type": "application/x-www-form-urlencoded",
               "Accept": "text/plain",
               "User-agent":"python-requests/"+version}

    # Encoding the request as a json string
    requestString = json.dumps(request)
    requestString = urlencode(requestString)
    
    # opening the https connection
    conn = httplib.HTTPSConnection(server)

    # Making the query
    conn.request("POST", "/api/v0/invoke", "request="+requestString, headers)

    # Getting the response
    resp = conn.getresponse()
    head = resp.getheaders()
    content = resp.read().decode('utf-8')

    # Close the https connection
    conn.close()

    return head,content

In [14]:
# Astroquery request function

def aqTicReq(tessid, column):
    requestTic = {"service":"Mast.Catalogs.Filtered.Tic", 
                  "format":"json",
                  "params":{
                      "columns":"c.{}".format(str(column)), #Column name to get
                      "filters":[
                           {"paramName":"ID",   #Filter by ID (tessid/ticid)
                             "values":[str(tessid)]}]
                        }
    }

    headers,outString = mastQuery(requestTic)
    outData = json.loads(outString)      #This isn't entirely necessary here
    return(outData['data'][0][str(column)])

In [15]:
# Planet name assembler

def nameResolver(ticid):
    request = {'service':'Mast.Name.Lookup',
                'params':{'input':str(ticid),
                          'format':'json'},
    }
     
    headers,outString = mastQuery(request)
     
    outData = json.loads(outString)
    
    return outData

def pnameGenerator(tceid):
    tessid, pl_number = tceid.split("-")
    tessid = str(int(tessid))
    pl_dict = {"01":"a", "02":"b", "03":"c", "04":"d", 
               "05":"e", "06":"f", "07":"g", "08":"h",
               "09":"i", "10":"j", "11":"k", "12":"l"}
    star_name = nameResolver(tessid)['resolvedCoordinate'][0]['canonicalName']
    return(str(star_name + "-" + pl_dict[pl_number]))


In [16]:
# test cell
nameResolver(218524525)

{'resolvedCoordinate': [{'searchString': '218524525',
   'resolver': 'K2',
   'cached': False,
   'resolverTime': 3,
   'searchRadius': 0.000333,
   'canonicalName': 'EPIC 218524525',
   'ra': 282.60174560546875,
   'decl': -18.646024703979492}],
 'status': ''}

In [20]:
# exoMAST info getters: urls

planeturl = "https://exo.mast.stsci.edu/api/v0.1/exoplanets/"
dvurl = "https://exo.mast.stsci.edu/api/v0.1/dvdata/tess/"
header_secinfo={}
header={}

In [21]:
# exoMAST info getters: sector getter

def getSector(tessid, get1=True, tce="TCE_1"):
    name = "TIC " + str(tessid)
    url = planeturl + "/identifiers/"
    params = {"name":name}
    r = requests.get(url=url, params=params, headers=header_secinfo)
    planet_names = r.json()
    #tce = planet_names['tessTCE']
    url = dvurl + str(tessid) + '/tces/'
    myparams = {"tce" : tce}

    r = requests.get(url = url, params = myparams, headers = header_secinfo)
    sectorInfo = r.json()

    sectors = [x[:11] for x in sectorInfo["TCE"] if tce in x][0][:5]
    sector_split = [x.split(":") for x in sectorInfo["TCE"] if tce in x][0][:5]
    if get1 == True:
        return(sectors)    
    else:
        return(sector_split)

In [22]:
#test cell
print(getSector(218524525))
#print(getSector(218524525, False, "TCE_2"))
print(getSector(218524525, False))

s0001
['s0001-s0009', 'TCE_1']


In [23]:


def tceConverter(tce):
    tce_dict = {"01":"TCE_1", "02":"TCE_2", "03":"TCE_3", "04":"TCE_4",
               "05":"TCE_5", "06":"TCE_6", "07":"TCE_7", "08":"TCE_8",
               "09":"TCE_9", "10":"TCE_10", "11":"TCE_11", "12":"TCE_12"}
    return(tce_dict[tce])

def getPlanetProp(tceid, prop_name):
    
    tceid = str(tceid)
    tessid, pl_number = tceid.split("-")
    tce = tceConverter(pl_number)
    
    tessid = str(int(tessid))
    
    star_name = "TIC " + str(tessid)
    
    sectors = getSector(tessid, False, tce)
    full_pname = (star_name + " (" + sectors[0].upper() + ") " + tce)
    
    url = planeturl + str(full_pname) + "/properties/"
    r = requests.get(url = url, headers = header)
    planet_prop = r.json()
    
    if planet_prop[0][prop_name] == None:
        return_prop = 0
    else:
        return_prop = planet_prop[0][prop_name]
    return(return_prop)
    

218524525 TCE_1
{'resolvedCoordinate': [{'searchString': '218524525', 'resolver': 'K2', 'cached': False, 'resolverTime': 1, 'searchRadius': 0.000333, 'canonicalName': 'EPIC 218524525', 'ra': 282.60174560546875, 'decl': -18.646024703979492}], 'status': ''}


In [376]:
# test cell

getPlanetProp("0218524525-01", 'constellation')
#tceConverter('01')

'Puppis'

In [24]:
#Stars = pd.DataFrame(columns=[
#    "star_tessid", "twomass",
#    "gaiaid", "sector",
#    "magnitude", "luminosity",
#    "star_mass", "star_radius",
#    "constellation", "disposition",
#    "rightascension", "declination"
#])
Planets = pd.DataFrame(columns=["planet_tce",
    "planetid", "star_tessid",
    "orbit_period", "planet_mass",
    "planet_mass", "planet_radius"
])

In [368]:
# Using the same format as the dummy data generator.
# Randoms will be replaced with indexing from various sources.

i = 0
for tessid in tessids:
    if tessid not in Stars['star_tessid'].values:
        print(tessid, i)
        star = {
            "star_tessid": int(tessid),
            "twomass": aqTicReq(tessid, 'TWOMASS'),
            "gaiaid": aqTicReq(tessid, 'GAIA'),
            "sector": getSector(tessid, True),
            "magnitude": aqTicReq(tessid, 'Tmag'),
            "luminosity": aqTicReq(tessid, 'lumclass'),
            "star_mass": aqTicReq(tessid, 'mass'),
            "star_radius": (tce_data.loc[tce_data['ticid'] == 
                                         tessid]['starRadiusSolarRadii'].values[0]),
            "constellation" : getPlanetProp(tce_data.
                                            loc[tce_data['ticid'] == tessid]['tceid']
                                           .values[0], "constellation"),
            "disposition": aqTicReq(tessid, 'disposition'),
            "rightascension": str(round(float(aqTicReq(tessid, 'ra')), 8)),
            "declination": str(round(float(aqTicReq(tessid, 'dec')), 8)),
            }
        Stars = Stars.append(star, ignore_index=True)
        i += 1

    



294329266 0
294394054 1
294395926 2
294399290 3
294750180 4
294754396 5
294780517 6
294784522 7
294873502 8
294922857 9
294978051 10
294981566 11
294983202 12
295071039 13
29684896 14
29697290 15
29699240 16
29779873 17
29780677 18
29781292 19
29829265 20
29830830 21
29831208 22
29924856 23
299802315 24
299833067 25
29983665 26
29984014 27
29986808 28
29987437 29
299897992 30
299899924 31
299906906 32
299940359 33
299942954 34
299945285 35
300010961 36
300012851 37
300013489 38
300013921 39
300015056 40
300033585 41
300033857 42
300037819 43
300038935 44
300039094 45
300039099 46
300136240 47
300136653 48
300137648 49
300159082 50
300159369 51
300160685 52
300161962 53
300241864 54
300243406 55
300243981 56
300290214 57
300290733 58
300290824 59
300293197 60
300325379 61
300326496 62
300327482 63
300329283 64
30034081 65
300379556 66
300380146 67
300443029 68
300443382 69
300447314 70
300448625 71
300448815 72
300449121 73
300560295 74
300561157 75
300561381 76
300601006 77
300705503 7

382187251 601
382188882 602
382200946 603
382256692 604
382256863 605
382258517 606
382259669 607
382259748 608
382266319 609
382268136 610
382302446 611
382437243 612
382510339 613
382512330 614
382517717 615
382517745 616
382519218 617
382528174 618
382529132 619
382529583 620
382575703 621
382575967 622
382577618 623
382579057 624
382624902 625
382626661 626
382630542 627
382631664 628
382631678 629
382632137 630
382959522 631
383134735 632
383664368 633
38396965 634
384031569 635
38459686 636
38460940 637
38461453 638
38462407 639
38462492 640
38469670 641
38469800 642
38507813 643
38508395 644
38509907 645
38510224 646
38515801 647
38517744 648
38518773 649
38518913 650
38570985 651
38571020 652
38571641 653
38572739 654
38572963 655
38573584 656
38573641 657
38574033 658
38574566 659
38583375 660
38585053 661
38586438 662
38587275 663
38602419 664
38603673 665
38633462 666
38634217 667
38634797 668
38635641 669
38635760 670
38678931 671
38679058 672
38680132 673
38696100 674
3869

In [ ]:
i = 0
for tce in tceids:
    if tce not in Planets['planet_tce'].values:
        print(tce, i)
        planet = {
            "planet_tce": tce,
            "planetid": pnameGenerator(tce),
            "star_tessid": int(tce_data.loc[tce_data['tceid'] == tce]['ticid'].values[0]),
            "orbit_period": float(tce_data.loc[tce_data['tceid'] == tce]['orbitalPeriodDays'].values[0]),
            "planet_mass": getPlanetProp(tce, 'Mp'),
            "planet_radius": float(tce_data.loc[tce_data['tceid'] == tce]['planetRadiusEarthRadii'].values[0]),
        }
        Planets = Planets.append(planet, ignore_index=True)
        i += 1

00101363142-01 0
00101364075-01 1
00101365484-01 2
00101395259-01 3
00101395750-01 4
00101395750-02 5
00101395750-03 6
00101417386-01 7
00101417386-02 8
00101417386-03 9
00117739806-01 10
00117739806-02 11
00118170779-01 12
00118307280-01 13
00118307280-02 14
00118313102-01 15
00118313102-02 16
00118313102-03 17
00118313102-04 18
00118313102-05 19
00118316147-01 20
00118414770-01 21
00118414770-02 22
00118417121-01 23
00118541883-01 24
00118541883-02 25
00118541883-03 26
00119292328-01 27
00119327278-01 28
00119327278-02 29
00119327278-03 30
00122330743-01 31
00122330743-02 32
00122907209-01 33
00122907209-02 34
00122907209-03 35
00123029030-01 36
00123029030-02 37
00123036723-01 38
00123153249-01 39
00123380233-01 40
00123382400-01 41
00123382400-02 42
00123480394-01 43
00123482865-01 44
00123699904-01 45
00123702439-01 46
00123744995-01 47
00123744995-02 48
00123744995-03 49
00124025288-01 50
00124025288-02 51
00124199788-01 52
00126430122-01 53
00126737992-01 54
00129268651-01 55
00

00149346418-01 437
00149346418-02 438
00149346418-03 439
00149346418-04 440
00149347480-01 441
00149347629-01 442
00149347629-02 443
00149347629-03 444
00149347805-01 445
00149351776-01 446
00149351776-02 447
00149392574-01 448
00149420896-01 449
00149420896-02 450
00149420896-03 451
00149423433-01 452
00149423433-02 453
00149423652-01 454
00149444545-01 455
00149444545-02 456
00149444545-03 457
00149444545-04 458
00149473467-01 459
00149502001-01 460
00149503144-01 461
00149539114-01 462
00149539114-02 463
00149539114-03 464
00149539114-04 465
00149539114-05 466
00149539842-01 467
00149539842-02 468
00149539894-01 469
00149542947-01 470
00149542947-02 471
00149542947-03 472
00149542947-04 473
00149542947-05 474
00149573437-01 475
00149573437-02 476
00149573437-03 477
00149573437-04 478
00149573659-01 479
00149573659-02 480
00149573659-03 481
00149600376-01 482
00149600376-02 483
00149603524-01 484
00149603524-02 485
00149603524-03 486
00149603524-04 487
00149625254-01 488
00149630117-

In [379]:
getPlanetProp("00218524525-01", 'planet_name')

'TIC 218524525 (S0001-S0009) TCE_1'

In [306]:
float(tce_data.loc[tce_data['tceid'] == tce]['planetRadiusEarthRadii'].values[0])

8.37513

In [350]:
getPlanetProp(str(f'{218524525:011}') + "-01", "constellation")

IndexError: list index out of range

In [353]:
getPlanetProp("00218524525-01", "constellation")

IndexError: list index out of range

In [ ]:
for tessid in tessids[:100]:
    name = nameResolver(tessid)['resolvedCoordinate'][0]['canonicalName']
    url = planeturl + "/identifiers/"
    params = {"name":name}
    r = requests.get(url=url, params=params, headers=header_secinfo)
    planet_names = r.json()
    tce = planet_names['tessTCE']
    url = dvurl + str(123482865) + '/tces/'
    myparams = {"tce" : tce}

    r = requests.get(url = url, params = myparams, headers = header)
    if planet_names['planetNames']

In [323]:
missing = []
for tessid in tessids:
    if tessid not in Stars['star_tessid'].values:
        missing.append(tessid)

In [351]:
tce_data.loc[tce_data['ticid'] == missing[0]]['tceid']

1076    00218524525-01
Name: tceid, dtype: object

In [386]:
Stars.to_csv(r'./data/Stars.csv')

In [2]:
Stars.head()

NameError: name 'Stars' is not defined

In [1]:
Planets.head()

NameError: name 'Planets' is not defined